#  <span style="color:orange">关联规则挖掘教程 (ARUL101)</span>

**使用版本: PyCaret 2.0** <br />
**更新时间: 2020年8月24**

# 1.0 教程简介
欢迎阅读关联规则挖掘的初级教程**(ARUL101)**. 本教程使用`pycaret.arules`模块来介绍异常检测的基本操作，面向的读者是PyCaret的新手。

在本教程中你会学到:


* **获取数据:** 如何从PyCaret中获取数据？
* **环境搭建:** 如何搭建关联规则挖掘所需的必要环境？
* **创建模型:** 如何创建模型并标注原始数据？
* **模型性能图:** 如何使用多种图像来分析模型的性能？

阅读时间 : 大约15分钟


# 1.1 安装 PyCaret
安装PyCaret一般只需要几分钟。请按照以下说明进行操作。

# 在本地 Jupyter Notebook 安装 PyCaret
`pip install pycaret`  <br />

# 在 Google Colab 或者 Azure Notebooks 安装 PyCaret
`!pip install pycaret`


# 1.2 预先要求
- Python 3.6 或更高
- PyCaret 2.0 或更高
- 网络链接来获取PyCaret中的数据
- 异常检测的基础知识

# 1.3 Google Colab 用户:
如何你在 Google Colab跑这个Notebook, 请运行以下代码来显示交互式的图像。<br/>
<br/>
`from pycaret.utils import enable_colab` <br/>
`enable_colab()`

# 2.0 什么是关联规则挖掘？
关联规则挖掘是一种基于规则的机器学习方法，用于发现大型数据中变量之间的有趣关系。它旨在使用一些有趣的度量来识别在数据中发现的强规则。例如，超市销售数据中的规则 {onions, potato} --> {burger} 表明如果客户同时购买洋葱和土豆，他们很可能也会购买汉堡。此类信息可用作营销活动决策，例如促销定价或产品展示位置。

__[阅读更多](https://en.wikipedia.org/wiki/Association_rule_learning)__

# 3.0 PyCaret 关联规则挖掘概述
PyCaret 的关联规则模块（`pycaret.arules`）是一个有监督的机器学习模块，用于发现数据集中变量之间的有趣关系。该模块自动将任何数据库转换为算法可接受的形状。 Apriori 是一种在关系数据库上进行频繁项集挖掘和关联规则学习的算法。

# 教程数据集

在本教程中，我们将使用来自 UCI 数据集的一个小样本，名为 **Online Retail Dataset**。这是一个交易数据集，其中包含 2010 年 1 月 12 日至 2011 年 9 月 12 日之间发生的交易，用于英国注册的非商店在线零售。公司主要销售独特的全场合礼品。该公司的许多客户都是批发商。属性的简短描述如下：


- **InvoiceNo:** 发票编号。名义上，一个 6 位整数，每笔交易唯一。如果此代码以字母“c”开头，则表示交易取消。
- **StockCode:** 产品代码，一个 5 位整数，唯一分配给每个不同的产品。
- **Description:** 产品名称。
- **Quantity:** 每笔交易的每个产品的数量。
- **InvoiceData:** 发票日期和时间。数字，生成每笔交易的日期和时间。
- **UnitPrice:** 单价，以英镑为单位的产品价格。
- **CustomerID:** 顾客号码，为每个客户唯一分配的 5 位整数。
- **Country:** 国家名称，每个客户所在国家的名称。

# 数据集作者与联系方式:
Dr Daqing Chen, Director: Public Analytics group. chend@lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.


 __[原始数据集链接](http://archive.ics.uci.edu/ml/datasets/online+retail)__ 

# 5.0 获取数据

你可以选择下载数据 （__[下载链接](https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression)__），并用pandas来加载 __[(pandas教程)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)__。 或者你可以使用 PyCaret的`get_data()` 函数来获取数据 （需要网络链接）。

In [1]:
from pycaret.datasets import get_data
data = get_data('france')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,12/1/2010 8:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,12/1/2010 8:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,12/1/2010 8:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,12/1/2010 8:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,12/1/2010 8:45,0.65,12583.0,France


**注意:** 如果你想手动下载数据，你要把数据的国家过滤为法国。

In [2]:
#查看数据的形状
data.shape

(8557, 8)

# 6.0 配置 PyCaret 环境

`setup()` 函数初始化 PyCaret的环境并且将交易数据转换为Apriori算法可接受的形状。这个函数需要三个必须的参数：pandas dataframe, `transaction_id`（用于pivot数据）和`item_id` （用于找寻规则）。通常下，这个参数是我们感兴趣的变量，你也可以使用`ignore_items` 来忽视将一些值用于创建规则。

In [3]:
from pycaret.arules import *

In [4]:
exp_arul101 = setup(data = data, 
                    transaction_id = 'InvoiceNo',
                    item_id = 'Description') 

Description,Value
session_id,7256
# Transactions,461
# Items,1565
Ignore Items,None


当`setup`运行完毕后，它会打印出一些重要信息：
- **# Transactions 交易数量:**  数据集中独立的交易数量。<br/>
<br/>
- **# Items 商品数:** 数据集中独立的商品数，用`description`来计算出。 <br/>
<br/>
- **Ignore Items 忽视商品:** 需要被忽视的商品，经常我们有一些非常明显的商品关系需要我们去无视掉。比如很多交易数据都有运费，这是个显而易见的关系。 <br/>

# 7.0 创建模型

创建关联规则模型很简单。`create_model()`不需要强制参数。它有 4 个可选参数，如下所示：

- **metric:** 用于评估规则是否重要的指标。默认值是confidence，其他指标有'support', 'lift', 'leverage', 'conviction'。 <br/>
<br/>
- **threshold:** 评估指标的最小阈值，通过 `metric` 参数决定规则是否重要。默认设置为`0.5`。 <br/>
<br/>
- **min_support:** 一个介于 0 和 1 之间的浮点数，用于对返回的项集的最小支持。默认值为`0.05`。 <br/>
<br/>
- **round:** 保留小数点数量。<br/>

In [5]:
model1 = create_model() #model created and stored in model1 variable.

In [6]:
print(model1.shape) #141 rules created.

(141, 9)


In [7]:
model1.head() #see the rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.0651,0.6746,0.0651,1.0000,1.4823,0.0212,inf
1,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER CUPS),0.0868,0.1171,0.0846,0.9750,8.3236,0.0744,35.3145
2,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.0868,0.1085,0.0846,0.9750,8.9895,0.0752,35.6616
3,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER CUPS),0.0716,0.1171,0.0694,0.9697,8.2783,0.0610,29.1345
4,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.0716,0.1085,0.0694,0.9697,8.9406,0.0617,29.4208


# 8.0 用 `ignore_items`来无视商品

在`model1`中，`JUMBO BAG WOODLAND ANIMALS`和`POSTAGE`非常明显。在接下来的实例中，我们会用`ignore_items` 来无视它。

In [8]:
exp_arul101 = setup(data = data, 
                    transaction_id = 'InvoiceNo',
                    item_id = 'Description',
                    ignore_items = ['POSTAGE']) 

Description,Value
session_id,6114
# Transactions,461
# Items,1565
Ignore Items,['POSTAGE']


In [9]:
model2 = create_model()

In [10]:
print(model2.shape) #notice how only 45 rules are created vs. 141 above.

(45, 9)


In [11]:
model2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER CUPS),0.0868,0.1171,0.0846,0.9750,8.3236,0.0744,35.3145
1,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.0868,0.1085,0.0846,0.9750,8.9895,0.0752,35.6616
2,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.1085,0.1171,0.1041,0.9600,8.1956,0.0914,22.0716
3,(CHILDRENS CUTLERY SPACEBOY ),(CHILDRENS CUTLERY DOLLY GIRL ),0.0586,0.0629,0.0542,0.9259,14.7190,0.0505,12.6508
4,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.1171,0.1085,0.1041,0.8889,8.1956,0.0914,8.0239


# 9.0 模型表现

In [12]:
plot_model(model2)

In [13]:
plot_model(model2, plot = '3d')